In [20]:
--1. Write function name: StudenID_ Func1 with parameter @mavt, return the sum of sl*giaban corresponding.
DROP FUNCTION dbo.Student_Func1;
IF EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'dbo.Student_Func1') AND type in (N'FN', N'IF', N'TF', N'FS', N'FT')) 
    DROP FUNCTION [dbo].Student_Func1
GO

CREATE FUNCTION dbo.Student_Func1(@mavt NVARCHAR(5))
RETURNS INT
WITH RETURNS NULL ON NULL INPUT
AS
BEGIN
    DECLARE @sum_vt INT;

    SELECT @sum_vt = SUM(hd.SL * hd.GiaBan)
    FROM [ABCCompany].[dbo].[CHITIETHOADON] hd
    WHERE hd.MaVT = @mavt;   

    RETURN @sum_vt;
END
GO

-- Test the function
PRINT dbo.Student_Func1(N'VT01');


Commands completed successfully.

Commands completed successfully.

7185000

Total execution time: 00:00:00.015

In [26]:
--2.Write  function to return a total of the HoaDon (@MahD is a parameter) 
IF EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'dbo.Student_Func2') AND type in (N'FN', N'IF', N'TF', N'FS', N'FT')) 
    DROP FUNCTION [dbo].Student_Func2
GO

CREATE FUNCTION dbo.Student_Func2(@MaHD NVARCHAR(10))
RETURNS INT
WITH RETURNS NULL ON NULL INPUT
AS
BEGIN
    DECLARE @sum_vt INT;

    SELECT @sum_vt = SUM(hd.SL * hd.GiaBan)
    FROM [ABCCompany].[dbo].[CHITIETHOADON] hd
    WHERE hd.MaHD = @MaHD;   

    RETURN @sum_vt;
END
GO

-- Test the function
PRINT dbo.Student_Func2(N'HD001');


Commands completed successfully.

Commands completed successfully.

560000

Total execution time: 00:00:00.003

In [44]:
--3. Write procedure name: StudenId _Proc1, parameter @makh, @diachi. This procedure help user update @diachi corresponding @makh.
IF EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'dbo.StudentId_Proc1') AND type in (N'P')) 
    DROP PROCEDURE [dbo].StudentId_Proc1
GO
CREATE PROCEDURE StudentId_Proc1
    @makh NVARCHAR(5),
    @diachi NVARCHAR(50)
AS
BEGIN
    BEGIN TRY
        UPDATE [ABCCompany].[dbo].[KHACHHANG]
        SET diachi = @diachi
        WHERE makh = @makh
    END TRY
    BEGIN CATCH
        PRINT ERROR_MESSAGE()
    END CATCH
END
--Test the function
GO
EXEC StudentId_Proc1 @makh = 'KH02', @diachi = 'QUY NHON'
SELECT * FROM [ABCCompany].[dbo].[KHACHHANG]

Commands completed successfully.

Commands completed successfully.

(1 row affected)

(6 rows affected)

Total execution time: 00:00:00.016

MaKH,TenKH,DiaChi,DT,Email
KH01,NGUYEN THI BE,QUAN 8,8457895,bnt@yahoo.com
KH02,LE HOANG NAM,QUY NHON,9878987,namlehoang @abc.com.vn
KH03,TRAN THI CHIEU,TAN BINH,8457895,NULL
KH04,MAI THI QUE ANH,BINH CHANH,NULL,NULL
KH05,LE VAN SANG,QUAN 10,NULL,sanglv@hcm.vnn.vn
KH06,TRAN HOANG KHAI,TAN BINH,8457897,NULL


In [47]:
--4.Write a procedure to add an item into HOADON
IF EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'dbo.StudentId_Proc2') AND type in (N'P')) 
    DROP PROCEDURE [dbo].StudentId_Proc2
GO
CREATE PROCEDURE StudentId_Proc2
    @MaHD NVARCHAR(10),
    @Ngay DATETIME,
    @MaKh NVARCHAR(5),
    @TongTG INT
AS
BEGIN
    BEGIN TRY
        INSERT INTO [ABCCompany].[dbo].[HOADON] (MaHD, Ngay, MaKH, TongTG)
        VALUES (@MaHD, @Ngay, @MaKh, @TongTG)
    END TRY
    BEGIN CATCH
        PRINT ERROR_MESSAGE()
    END CATCH
END
--Test the function
GO
EXEC StudentId_Proc2 @MaHD = 'HD02', @Ngay = '2023-05-12 00:00:00.000', @makh = 'KH02', @TongTG = NULL
SELECT * FROM [ABCCompany].[dbo].[HOADON]

Commands completed successfully.

Commands completed successfully.

(1 row affected)

(11 rows affected)

Total execution time: 00:00:00.037

MaHD,Ngay,MaKH,TongTG
HD001,2000-05-12 00:00:00.000,KH01,NULL
HD002,2000-05-25 00:00:00.000,KH02,NULL
HD003,2000-05-25 00:00:00.000,KH01,NULL
HD004,2000-05-25 00:00:00.000,KH04,NULL
HD005,2000-05-26 00:00:00.000,KH04,NULL
HD006,2000-05-02 00:00:00.000,KH03,NULL
HD007,2000-06-22 00:00:00.000,KH04,NULL
HD008,2000-06-25 00:00:00.000,KH03,NULL
HD009,2000-08-15 00:00:00.000,KH04,NULL
HD010,2000-08-30 00:00:00.000,KH01,NULL


In [65]:
--5.Write trigger name: StudenId_Trigger on table Chitiethoadon, when user insert data into Chitiethoadon, the trigger will update the Tongtien in HoaDon(student should add Tongtien column into Hoadon, tongtien=sum(sl*giaban).
IF COL_LENGTH('ABCCompany.dbo.HOADON','TongTien') IS NULL
BEGIN
    ALTER TABLE [ABCCompany].[dbo].[HOADON]
    ADD TongTien DECIMAL(18, 2) NULL
END
GO

IF OBJECT_ID('dbo.StudentId_Trigger','TR') IS NOT NULL
    DROP TRIGGER dbo.StudentId_Trigger;
GO

CREATE TRIGGER StudentId_Trigger
ON [ABCCompany].[dbo].[CHITIETHOADON]
AFTER INSERT
AS 
BEGIN
    UPDATE [ABCCompany].[dbo].[HOADON]
    SET TongTien = (
        SELECT SUM(SL * GiaBan)
        FROM [ABCCompany].[dbo].[CHITIETHOADON] CTHD
        WHERE CTHD.MaHD = [ABCCompany].[dbo].[HOADON].MaHD)
    FROM [ABCCompany].[dbo].[HOADON]
    INNER JOIN inserted ON [ABCCompany].[dbo].[HOADON].MaHD = inserted.MaHD;
END



Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.003

In [ ]:
INSERT INTO [ABCCompany].[dbo].[CHITIETHOADON] (MaHD, MaVT, SL, KhuyenMai, GiaBan)
VALUES 
    ('HD001', 'VT06', 10, 0, 500),  
    ('HD002', 'VT06', 5, 10, 700),   
    ('HD003', 'VT06', 8, 5, 600);     


In [71]:
SELECT * FROM HOADON

(11 rows affected)

Total execution time: 00:00:00.011

MaHD,Ngay,MaKH,TongTG,TongTien
HD001,2000-05-12 00:00:00.000,KH01,NULL,565000.00
HD002,2000-05-25 00:00:00.000,KH02,NULL,1503500.00
HD003,2000-05-25 00:00:00.000,KH01,NULL,1104800.00
HD004,2000-05-25 00:00:00.000,KH04,NULL,NULL
HD005,2000-05-26 00:00:00.000,KH04,NULL,NULL
HD006,2000-05-02 00:00:00.000,KH03,NULL,NULL
HD007,2000-06-22 00:00:00.000,KH04,NULL,NULL
HD008,2000-06-25 00:00:00.000,KH03,NULL,NULL
HD009,2000-08-15 00:00:00.000,KH04,NULL,NULL
HD010,2000-08-30 00:00:00.000,KH01,NULL,NULL


In [74]:
--6.Write view name: StudentID_View1 to extract list of customers who bought ‘Gach Ong’
IF EXISTS(select * FROM sys.views where name = 'StudentID_View1')
    DROP VIEW StudentID_View1
GO

CREATE VIEW StudentID_View1 AS
SELECT DISTINCT KH.*
FROM [ABCCompany].[dbo].[CHITIETHOADON] CTHD
INNER JOIN [ABCCompany].[dbo].[HOADON] HD ON CTHD.MaHD = HD.MaHD
INNER JOIN [ABCCompany].[dbo].[KHACHHANG] KH ON HD.MaKH = KH.MaKH
INNER JOIN [ABCCompany].[dbo].[VATTU] VT ON VT.MaVT = CTHD.MaVT
WHERE VT.TenVT = N'Gach Ong'
GO

SELECT * FROM StudentID_View1

Commands completed successfully.

Commands completed successfully.

(2 rows affected)

Total execution time: 00:00:00.033

MaKH,TenKH,DiaChi,DT,Email
KH02,LE HOANG NAM,QUY NHON,9878987,namlehoang @abc.com.vn
KH04,MAI THI QUE ANH,BINH CHANH,NULL,NULL
